In [1]:
import sys

import socket
import traceback
from time import sleep
import threading

import cv2
from imutils.video import VideoStream
import imagezmq
import simplejpeg


In [2]:
%run utils.ipynb

In [3]:
# Helper class implementing an IO deamon thread
class VideoStreamSubscriber:

    def __init__(self, hostname, port):
        self.hostname = hostname
        self.port = port
        self._stop = False
        self._data_ready = threading.Event()
        self._thread = threading.Thread(target=self._run, args=())
        self._thread.daemon = True
        self._thread.start()

    def receive(self, timeout=15.0):
        flag = self._data_ready.wait(timeout=timeout)
        if not flag:
            raise TimeoutError(
                "Timeout while reading from subscriber tcp://{}:{}".format(self.hostname, self.port))
        self._data_ready.clear()
        return self._data

    def _run(self):
        receiver = imagezmq.ImageHub("tcp://{}:{}".format(self.hostname, self.port), REQ_REP=False)
        while not self._stop:
            self._data = receiver.recv_jpg()
            self._data_ready.set()
        receiver.close()

    def close(self):
        self._stop = True
        
        
jpeg_quality = 80
pub_name = socket.gethostname()


port = 5556
sender = imagezmq.ImageSender("tcp://*:{}".format(port), REQ_REP=False)

if __name__ == "__main__":
    # Receive from broadcast
    # There are 2 hostname styles; comment out the one you don't need
    hostname = "127.0.0.1"  # Use to receive from localhost
    # hostname = "192.168.86.38"  # Use to receive from other computer
    port = 5555
    receiver = VideoStreamSubscriber(hostname, port)

    try:
        counter = 0
        while True:
            display.clear_output(wait=True)
            msg, jpg_buffer = receiver.receive()
            image = simplejpeg.decode_jpeg(jpg_buffer, colorspace='BGR', fastdct=False, fastupsample=False)
            
            # image processing, pass
            image = cv2.resize(image, (400, 400), interpolation=cv2.INTER_CUBIC)

            
            jpg_buffer = simplejpeg.encode_jpeg(image, quality=jpeg_quality, colorspace='BGR', fastdct=False)
            sender.send_jpg(pub_name, jpg_buffer)
            print(f"{pub_name} sent frame {counter}")
            counter = counter + 1

    except (KeyboardInterrupt, SystemExit):
        print('Exit due to keyboard interrupt')
    except Exception as ex:
        print('Python error with no Exception handler:')
        print('Traceback error:', ex)
        traceback.print_exc()
    finally:
        receiver.close()
        sys.exit()

Exit due to keyboard interrupt


SystemExit: 

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
